In [ ]:
# Find and reference Input csv File from Uber folder

# P: List of files to read
files_name = [
    "other-American_B01362",
    "other-Carmel_B00256",
    "other-Diplo_B01196",
    "other-Federal_02216",
    "other-Firstclass_B01536",
    "other-Highclass_B01717",
    "other-Prestige_B01338",
    "other-Skyline_B00111"    
]

# P: Shortcut name to refer to each file
key_name = [
    "American",
    "Carmel",
    "Diplo",
    "Federal",
    "Firstclass",
    "Highclass",
    "Prestige",
    "Skyline"    
]

# P: The 9 files will be saved in 1 dictionary (uber_dict). The key for each file will be the shortcut name saved in
#    in the list 'key_name'. Example: To display the data frame of the file "other-Firstclass_B01536.csv" type:
#    uber_dict["Firstclass"]

uber_dict = {}

for index, file in enumerate(files_name):

    file_path = f"Uber_Rides_Rawdata/Uber/{file}.csv"

# P: I found that the files have different encoder so I had to use the try-except functions.
    try:
        uber_dict[key_name[index]] = pd.read_csv(file_path, encoding = "utf-8")
        print(f"GOOD | FILE: {file} | ENCODING: utf-8")
    except UnicodeDecodeError:
        try:
            uber_dict[key_name[index]] = pd.read_csv(file_path, encoding = "cp1252")
            print(f"GOOD | FILE: {file} | ENCODING: cp1252")
        except:
            print("ERROR")

print("Columns before cleaning\n")
for key in uber_dict:
    for index, column in enumerate(uber_dict[key]):
        print(key, index, column)

column_date = "date"
column_time = "TIME"
column_pickup = "PU_ADDRESS"


for key in uber_dict:
    for index, column in enumerate(uber_dict[key]):
        if index == 0:
            uber_dict[key] = uber_dict[key].rename(columns = {column: column_date})
        elif index == 1:
            uber_dict[key] = uber_dict[key].rename(columns = {column: column_time})
        elif index == 2:
            uber_dict[key] = uber_dict[key].rename(columns = {column: column_pickup})

    uber_dict[key] = uber_dict[key][[column_date, column_time, column_pickup]]
    uber_dict[key][column_date] = pd.to_datetime(uber_dict[key][column_date])

print("\n\nColumns after cleaning\n")
for key in uber_dict:
    for index, column in enumerate(uber_dict[key]):
        print(key, index, column)

for index, key in enumerate(uber_dict):
    if index == 0:

        merged_df = uber_dict[key].groupby(column_date).count().copy().reset_index()
    else:
#         pass
        merged_df = merged_df.merge(uber_dict[key].groupby(column_date).count().reset_index(), on = column_date, how = "outer", suffixes = (f"_{key_name[index-1]}", f"_{key_name[index]}"))

print(merged_df)

# merged_df = merged_df.set_index(column_date)
for column in merged_df:
    for value in key_name:
        if column == f"{column_pickup}_{value}":
            merged_df = merged_df.drop(column, axis = 1)
        elif column == f"{column_time}_{value}":
            merged_df = merged_df.rename(columns = {column: f"count_{value}"})


merged_df = merged_df.fillna(0)
merged_df["count_Federal"] = merged_df["count_Federal"].astype(int)
print(merged_df)


# UBER#########################################


# Find and reference Input csv File from Uber folder

# P: List of files to read
uberfiles_name = [
    "uber-raw-data-jul14",
    "uber-raw-data-aug14",
    "uber-raw-data-sep14",    
]

for index, file in enumerate(uberfiles_name):
    
    if index == 0:
        file_path = f"Uber_Rides_Rawdata/Uber/{file}.csv"

        # P: I found that the files have different encoder so I had to use the try-except functions.
        try:
            uber_df = pd.read_csv(file_path, encoding = "utf-8")
            print(f"GOOD | FILE: {file} | ENCODING: utf-8")
        except UnicodeDecodeError:
            try:
                uber_df = pd.read_csv(file_path, encoding = "cp1252")
                print(f"GOOD | FILE: {file} | ENCODING: cp1252")
            except:
                print("ERROR")
    else:
        file_path = f"Uber_Rides_Rawdata/Uber/{file}.csv"

        # P: I found that the files have different encoder so I had to use the try-except functions.
        try: 
            uber_df = uber_df.append(pd.read_csv(file_path, encoding = "utf-8"))
            print(f"GOOD | FILE: {file} | ENCODING: utf-8")
        except UnicodeDecodeError:
            try:
                uber_df = uber_df.append(pd.read_csv(file_path, encoding = "cp1252"))
                print(f"GOOD | FILE: {file} | ENCODING: cp1252")
            except:
                print("ERROR")
uber_df

In [ ]:
uber_df = uber_df.rename(columns = {uber_df.columns[0]: column_date})

# Line to split date and time
uber_df[[column_date, column_time]] = uber_df[column_date].str.split(" ", 1, expand = True)

uber_df[column_date] = pd.to_datetime(uber_df[column_date])
uber_df

In [ ]:
uber_df = uber_df[[column_date, column_time]]
count_uber = uber_df.groupby(column_date).count().reset_index()
count_uber = count_uber.rename(columns = {count_uber.columns[1]: "count_Uber"})
count_uber

In [ ]:
merged_df = merged_df.merge(count_uber, on = column_date, how = "outer")

In [ ]:
merged_df = merged_df.set_index(column_date)
merged_df["Total"] = merged_df.sum(axis = 1)
merged_df = merged_df.reset_index()
merged_df

In [ ]:
# P: Saving rides data (merged_df) to a csv file

merged_df.to_csv("Data/rides_data.csv", encoding = "utf-8", index = False, header = True)

In [ ]:
weather_rides_df = merged_df.copy()
weather_rides_df = weather_rides_df.merge(weather_data, left_on = "date", right_on = column_date, how = "outer")

weather_rides_df["Weekday"] = weather_rides_df[column_date].dt.weekday

days = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
days_list = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weather_rides_df["Weekday"] = weather_rides_df["Weekday"].map(days)
weather_rides_df
        

In [ ]:
# P: Saving merged df (weather and rides) to a csv file

weather_rides_df.to_csv("Data/weather_rides_df.csv", encoding = "utf-8", index = False, header = True)

In [ ]:
groupby_weekday_sum = weather_rides_df[["count_American", "count_Carmel", "count_Diplo", "count_Federal", "count_Firstclass", "count_Highclass", "count_Prestige", "count_Skyline", "count_Uber", "Total", "Weekday"]].groupby("Weekday").sum().reset_index()
# print(groupby_weekday_sum)
groupby_weekday_average = weather_rides_df[["maxtempC", "avgtempC", "uvIndex", "precipMM", "cloudcover", "humidity", "weatherDesc", "Weekday"]].groupby("Weekday").mean().reset_index()
# print(groupby_weekday_average)

weekday_df = groupby_weekday_sum.copy()
weekday_df = weekday_df.merge(groupby_weekday_average, on = "Weekday", how = "outer", suffixes = (f"_sum", f"_mean")).set_index("Weekday")
weekday_df = weekday_df.reindex(days_list)

for index, column in enumerate(weekday_df):
#     print(index)
    if index <= 9:
        if index == 9:
            weekday_df = weekday_df.rename(columns = {column: f"sum_Total"})
        elif index == 8:
            weekday_df = weekday_df.rename(columns = {column: f"sum_Uber"})
        else:
            weekday_df = weekday_df.rename(columns = {column: f"sum_{key_name[index]}"})
    else:
        weekday_df = weekday_df.rename(columns = {column: f"avg_{column}"})


weekday_df.to_csv("Data/weekday_df.csv", encoding = "utf-8", index = True, header = True)

print(weekday_df)